In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

#DATASET PREPERATION

In [ ]:
!pip install pandas
!pip install xlrd==1.2.0
!pip install json
!pip install re
!pip install requests
!pip install gdown
!pip install emoji
!pip install clean-text[gpl]

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=60d0563ef0376039e4c6f40333264a344e07e374a3126b62e897e3ddc1c5402e
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.9.0
    Uni

In [ ]:
import pandas as pd
import json
import requests
import re
from cleantext import clean

In [ ]:
file = 'path/to/xlsx'
!gdown $file

Downloading...
From: https://drive.google.com/u/0/uc?id=1JcESl7qCCBvS6xpWMZplhCXunvmkcNU_
To: /content/behaviour_simulation_train.xlsx
100% 56.2M/56.2M [00:00<00:00, 183MB/s]


In [ ]:
data_xl = pd.read_excel("behaviour_simulation_train.xlsx")

In [ ]:
data_xl.to_csv('train.csv')

In [ ]:

data = pd.read_csv('train.csv', index_col= 0)

/bin/bash: line 1: -o: command not found
--2023-12-13 10:38:37--  https://drive.google.com/u/0/uc?id=1-2lyZ6G3S2l9JUj0gwsB_JI-2sv0lZ6N
Resolving drive.google.com (drive.google.com)... 173.194.193.139, 173.194.193.101, 173.194.193.100, ...
Connecting to drive.google.com (drive.google.com)|173.194.193.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1-2lyZ6G3S2l9JUj0gwsB_JI-2sv0lZ6N [following]
--2023-12-13 10:38:37--  https://drive.google.com/uc?id=1-2lyZ6G3S2l9JUj0gwsB_JI-2sv0lZ6N
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/usa13ghm5egsmtpc4oig11rbn1uuokq4/1702463850000/11072730283631302471/*/1-2lyZ6G3S2l9JUj0gwsB_JI-2sv0lZ6N?uuid=2ceda6fb-b275-4fac-85cf-b0fcc5b4184e [following]
--2023-12-13 10:38:40--  https://doc-0c-88-docs.googleusercontent.com/docs/secures

<h1>Tasks</h1>
<ol>
<li> create columns url, views, duration, status, and drop media column
<li> for videos, extract "lowest bitrate" "available" mp4 format file,
<li> for gif, extract any mp4 format file
<li> for image, extract "first" "full url" "available" jpg/png format file
<li> duration -1 == None
<li> Views -1 == None
<li> around 50k media files are offline(need to recheck)
</ol>

In [ ]:
# these functions directly work with data['media'] values

# for getting lowest res video
def get_video_url(media):
  pattern = r"contentType='video/mp4', url='(.*?)', bitrate=(\d+)"
  matches = re.findall(pattern, media)
  lowest_bitrate_url = min(matches, key=lambda x: int(x[1]))[0]
  # url = re.sub(r'\.mp4.*$', '.mp4',lowest_bitrate_url)
  url = lowest_bitrate_url
  return url

# for getting duration and views
def get_duration_views(text):
    pattern = r'duration=([0-9.]+),\s*views=(\d+|None)'
    match = re.search(pattern, text)
    if match:
        duration = float(match.group(1))
        views = int(match.group(2)) if match.group(2) != 'None' else -1
        return duration, views
    else:
        print(text)
        return None

# for first image url and its format
def get_first_image_url_format(media):
  pattern = r"fullUrl='(.*?)'"
  matches = re.findall(pattern, media)
  first_image = matches[0]
  format = re.findall(r"format=([^&]+)", first_image)[0]
  return first_image, format

# for first image url (http\\:askdhaks.jpg)
def get_first_image_url(media):
  full_url, format = get_first_image_url_format(media)
  # pattern = r"(.*?)\?"
  # matches = re.findall(pattern, full_url)
  # return matches[0]+f".{format}"
  return full_url

# for checking status
def get_status(url):
  res = requests.head(url, timeout=5)
  return res.status_code

In [ ]:
def media_to_features(media):
    mtype = ""
    url = ""
    duration = -1
    views = -1

    if "[Gif(" in media:
        mtype = "Gif"
        url = get_video_url(media)
    elif "[Video(" in media:
        mtype = "Video"
        url = get_video_url(media)
        duration, views = get_duration_views(media)

    elif "[Photo(" in media:
        mtype = "Photo"
        url = get_first_image_url(media)

    # result["status"] = get_status(url)

    return mtype, url, duration, views

In [ ]:
mask1 = data['media'].str.contains("\[Video\(")
mask2 = data['media'].str.contains("\[Photo\(")
mask3 = data['media'].str.contains("\[Gif\(")

filtered_df1 = data[mask1]
filtered_df2 = data[mask2]
filtered_df3 = data[mask3]

print("number of videos: ", len(filtered_df1))
print("number of images: ", len(filtered_df2))
print("number of gifs  : ", len(filtered_df3))
print("total number    : ", len(data))


number of videos:  77146
number of images:  210009
number of gifs  :  12845
total number    :  300000


In [ ]:
jpg = 0
png = 0
other = 0

for media in filtered_df2.media.values:
  _,format = get_first_image_url_format(media)
  if (format=='jpg'):
    jpg+=1
  elif (format=='png'):
    png+=1
  else:
    other+=1
print("images in jpg  :", jpg)
print("images in png  :", png)
print("images in other:", other)
print("total number   :", len(filtered_df2))

images in jpg  : 199729
images in png  : 10280
images in other: 0
total number   : 210009


In [ ]:
text = filtered_df2.sample(1).media.values[0]
print(get_first_image_url(text))

https://pbs.twimg.com/media/EbsA7CpWAAAdw2E?format=jpg&name=large


In [ ]:
data[['type', 'url', 'duration', 'views']] = data['media'].apply(media_to_features).apply(pd.Series)

In [ ]:
data.sample(5)

,id,date,likes,content,username,media,inferred company,type,url,duration,views
292259,292260,2020-04-26 14:32:33,0,Attacking Umahi Will Not Distract Him From Dev...,IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,Photo,https://pbs.twimg.com/media/EWiWmuMU8AAZsL2?fo...,-1.0,-1
110798,110799,2020-08-27 04:45:01,1,E.ON took from 2 hours to just 5 minutes in de...,cisco_in,[Photo(previewUrl='https://pbs.twimg.com/media...,cisco,Photo,https://pbs.twimg.com/media/EgZrriXWsAAHG8Q?fo...,-1.0,-1
164345,164346,2018-05-20 12:33:41,94,The #GangOfDusters explores the rich cultural ...,RenaultIndia,[Photo(previewUrl='https://pbs.twimg.com/media...,renault,Photo,https://pbs.twimg.com/media/Ddo_O64V4AAiJ2D?fo...,-1.0,-1
229370,229371,2019-11-06 14:25:04,0,Man Remanded Over Alleged Rape Of 14-Year-Old ...,IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,Photo,https://pbs.twimg.com/media/EIsjcORVAAAtEfA?fo...,-1.0,-1
94882,94883,2019-12-05 04:18:17,2,And that's a wrap to an eventful Community Imp...,cisco_in,[Photo(previewUrl='https://pbs.twimg.com/media...,cisco,Photo,https://pbs.twimg.com/media/EK_uqjyWoAAXs7X?fo...,-1.0,-1


In [ ]:
df = data.drop(['media'], axis = 1)

In [ ]:
df.to_csv('train_extended.csv')

In [ ]:
# saving work
# from google.colab import drive
# drive.mount('/content/drive')
# !cp train_extended.csv /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
min_length_url = min(df['url'], key=len)

print("Minimum length URL:", min_length_url)
print("Minimum length:", len(min_length_url))

Minimum length URL: https://video.twimg.com/tweet_video/EPFYz1vXUAE7jdr.mp4
Minimum length: 55


In [ ]:
df.sample(5)

In [ ]:
# taking 10 hours
# import concurrent.futures
# from tqdm import tqdm

# with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
#     results = list(tqdm(executor.map(get_status, df['url']), total=len(df)))

# df['status_code'] = results

# loading

In [ ]:
# !gdown path/to/converted/csv/xlsx --fuzzy -O train_extended.csv
data = pd.read_csv('train_extended.csv', index_col= 0)

In [ ]:
min_length = data['content'].str.len().min()
max_length = data['content'].str.len().max()
print("min content", min_length)
print("max content", max_length)

min content 20
max content 540


In [ ]:
print(data.sample(10).content.values)

['Showing my little family around my beloved Durham reminiscing about uni days 💕 <hyperlink>'
 'Brian Burns got hands\n\nHi <mention> 👋 <hyperlink>'
 'Today, tomorrow, and every day we honor and thank our veterans who defend, and protect our inalienable rights.  Grateful to you and for you. Thank you . #VeteransDay 🇺🇸 <hyperlink>'
 'I like these guys a lot. They stood in line to meet without muscling their way ahead (dudes skipping women in line after diversity-related talks happens more than you’d think🤦🏾\u200d♀️), had such positive energy, &amp; seem like fantastic allies. Just found this post of theirs. <hyperlink>'
 'Afraid of #CyberAttacks? <mention>’s R&amp;D center in the Middle East isn’t, thanks to the Defense in depth concept from Siemens! Curious? Join us at #HM18 to get more info: <hyperlink> <hyperlink>'
 'Three people ticketed for failing to self-isolate after arriving in Nova Scotia <hyperlink> <hyperlink>'
 'i am a brand. follow me. <hyperlink>'
 'Just clicked on the sk

In [ ]:
!pip install emoji
!pip install clean-text[gpl]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
from cleantext import clean
def remove_emojis_and_tags(text):
    # Remove emojis

    clean_text = clean(text,
        fix_unicode=True,               # fix various unicode errors
        to_ascii=True,                  # transliterate to closest ASCII representation
        lower=True,                     # lowercase text
        no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_phone_numbers=False,         # replace all phone numbers with a special token
        no_numbers=False,               # replace all numbers with a special token
        no_digits=False,                # replace all digits with a special token
        no_currency_symbols=False,      # replace all currency symbols with a special token
        no_punct=False,                 # remove punctuations
        no_emoji=True,
        # replace_with_punct="",          # instead of removing punctuations you may replace them
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        # replace_with_phone_number="<PHONE>",
        # replace_with_number="<NUMBER>",
        # replace_with_digit="0",
        # replace_with_currency_symbol="<CUR>",
        lang="en"                       # set to 'de' for German special handling
    )

    # Remove text within <>
    text_no_tags = ' '.join(word for word in clean_text.split() if not (word.startswith('<') and word.endswith('>')))

    return text_no_tags

In [ ]:
data['content'] = data['content'].apply(remove_emojis_and_tags)

In [ ]:
min_length = data['content'].str.len().min()
max_length = data['content'].str.len().max()
print("min content", min_length)
print("max content", max_length)

min content 4
max content 300


In [ ]:
print(data.sample(10).content.values)

['pull cash seamlessly from your bank to your airtel money wallet at no extra cost. dial *185*7*1# to get started on the transfer. #besmartbesafe'
 'happening now in room 501/502 - kops for #openstack presented by derek lemon. #openstack #openinfrastructuresummit #kubernetes #ops'
 '"not all country boys are bigots... happy pride month." this man\'s message of acceptance from rural oklahoma touched people around the world. he tells cnn why he finds it so important to be an ally:'
 '#dublin fog affecting visibility on the m50 and n4 at the moment. take care. more here:'
 'your #epoty2019 is... stalwart consistent, incisive, a stand out all season #heineken #championscup'
 'from the menu - a dessert of blueberry and walnuts with ants and wood sorrel!'
 "what are the key criteria to choose a suitable #iot #platform? how can iot #applications be monetized using the right platform? #mindsphere coo explains you the dos and don'ts in this webinar with"
 "how has #covid19 impacted conservation

In [ ]:
data.duration.describe()

count    300000.000000
mean         15.996764
std          47.886853
min          -1.000000
25%          -1.000000
50%          -1.000000
75%           5.000000
max         599.933000
Name: duration, dtype: float64

In [ ]:
data['duration'] = data['duration'].apply(lambda x: max(1, x)).astype(int)

In [ ]:
data.duration.describe()

count    300000.000000
mean         17.372250
std          47.241855
min           1.000000
25%           1.000000
50%           1.000000
75%           5.000000
max         599.000000
Name: duration, dtype: float64

In [ ]:
print("duration of videos in dataset\n",data[data['type'] == 'Video']['duration'][data['duration'] >= 0].describe())

duration of videos in dataset
 count    77146.000000
mean        64.667267
std         75.284399
min          1.000000
25%         16.000000
50%         41.000000
75%         81.000000
max        599.000000
Name: duration, dtype: float64


In [ ]:
print("views of videos in dataset\n",data[data['type'] == 'Video']['views'][data['views'] >= 0].describe())
mean_pos_views = int(data[data['type'] == 'Video']['views'][data['views'] >= 0].mean())
# mean values as views as views of images and gifs
data['views'] = data['views'].replace(-1, mean_pos_views).astype(int)

views of videos in dataset count    7.712700e+04
mean     1.525633e+05
std      1.363441e+06
min      1.000000e+00
25%      2.945000e+03
50%      1.109900e+04
75%      5.864800e+04
max      2.230419e+08
Name: views, dtype: float64


In [ ]:
data['date'] = pd.to_datetime(data['date']).dt.date.astype(str)

In [ ]:
data.sample(5)

,id,date,likes,content,username,inferred company,type,url,duration,views
188965,188966,2018-06-20,729,watch the 19-year-old royal wedding cellist pl...,CNN,cnn,Video,https://video.twimg.com/amplify_video/10091066...,94,61264
196580,196581,2020-01-17,33,introducing smarter security. it's human natur...,BlackBerry,blackberry,Photo,https://pbs.twimg.com/media/EOgHJOfX0AALhUs?fo...,1,152563
37019,37020,2019-02-26,676,kicking off the second week of testing with be...,WilliamsRacing,williams,Photo,https://pbs.twimg.com/media/D0UUPGEW0AEe2V-?fo...,1,152563
165720,165721,2020-07-09,1,"most efficient #opelastra of all time, with th...",OpelIreland,opel,Video,https://video.twimg.com/amplify_video/12812336...,9,37
286771,286772,2020-05-07,1,covid-19: wear face mask or risk 14 days isola...,IndependentNGR,independent,Photo,https://pbs.twimg.com/media/EXaaGjfU8AAQGgE?fo...,1,152563


In [ ]:
data.to_csv('train_clean.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp train_clean.csv /content/drive/MyDrive

Mounted at /content/drive


# test data preprocessing

In [ ]:
!gdown link/to/test/files
!gdown link/to/test/files
!gdown link/to/test/files
!gdown link/to/test/files

In [ ]:
behaviour_simulation_test_company = pd.read_excel("behaviour_simulation_test_company.xlsx")
behaviour_simulation_test_time = pd.read_excel("behaviour_simulation_test_time.xlsx")
content_simulation_test_time = pd.read_excel("content_simulation_test_time.xlsx")
content_simulation_test_company = pd.read_excel("content_simulation_test_company.xlsx")

In [ ]:
# behaviour_simulation_test_company.head() # id date username content media ic
# behaviour_simulation_test_time.head() # id date username content media ic
# content_simulation_test_time.head() # no content
content_simulation_test_company.head() # no content

,id,date,likes,username,media,inferred company
0,1,2019-05-10 11:44:04,2,UPMCnews,[Photo(previewUrl='https://pbs.twimg.com/media...,upmc
1,2,2019-09-03 14:00:01,1,rbcgamnews,[Photo(previewUrl='https://pbs.twimg.com/media...,rbc
2,3,2018-12-02 05:51:16,502,BBCSport,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc
3,4,2018-08-04 20:03:17,151,BBCSport,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc
4,5,2019-02-11 04:29:29,362,uclagymnastics,[Photo(previewUrl='https://pbs.twimg.com/media...,ucla


In [ ]:
behaviour_simulation_test_company[['type', 'url', 'duration', 'views']] = behaviour_simulation_test_company['media'].apply(media_to_features).apply(pd.Series)
behaviour_simulation_test_time[['type', 'url', 'duration', 'views']] = behaviour_simulation_test_time['media'].apply(media_to_features).apply(pd.Series)
content_simulation_test_time[['type', 'url', 'duration', 'views']] = content_simulation_test_time['media'].apply(media_to_features).apply(pd.Series)
content_simulation_test_company[['type', 'url', 'duration', 'views']] = content_simulation_test_company['media'].apply(media_to_features).apply(pd.Series)

In [ ]:
behaviour_simulation_test_company = behaviour_simulation_test_company.drop(['media'], axis = 1)
behaviour_simulation_test_time = behaviour_simulation_test_time.drop(['media'], axis = 1)
content_simulation_test_time = content_simulation_test_time.drop(['media'], axis = 1)
content_simulation_test_company = content_simulation_test_company.drop(['media'], axis = 1)

In [ ]:
behaviour_simulation_test_company['content'] = behaviour_simulation_test_company['content'].apply(remove_emojis_and_tags)
behaviour_simulation_test_time['content'] = behaviour_simulation_test_time['content'].apply(remove_emojis_and_tags)
# content_simulation_test_time['content'] = content_simulation_test_time['content'].apply(remove_emojis_and_tags)
# content_simulation_test_company['content'] = content_simulation_test_company['content'].apply(remove_emojis_and_tags)


In [ ]:
behaviour_simulation_test_company['duration'] = behaviour_simulation_test_company['duration'].apply(lambda x: max(1, x)).astype(int)
behaviour_simulation_test_time['duration'] = behaviour_simulation_test_time['duration'].apply(lambda x: max(1, x)).astype(int)
content_simulation_test_time['duration'] = content_simulation_test_time['duration'].apply(lambda x: max(1, x)).astype(int)
content_simulation_test_company['duration'] = content_simulation_test_company['duration'].apply(lambda x: max(1, x)).astype(int)

mean_pos_views = 1.525633e+05 # from train
# mean values as views as views of images and gifs
behaviour_simulation_test_company['views'] = behaviour_simulation_test_company['views'].replace(-1, mean_pos_views).astype(int)
behaviour_simulation_test_time['views'] = behaviour_simulation_test_time['views'].replace(-1, mean_pos_views).astype(int)
content_simulation_test_time['views'] = content_simulation_test_time['views'].replace(-1, mean_pos_views).astype(int)
content_simulation_test_company['views'] = content_simulation_test_company['views'].replace(-1, mean_pos_views).astype(int)

In [ ]:
behaviour_simulation_test_company['date'] = pd.to_datetime(behaviour_simulation_test_company['date']).dt.date.astype(str)
behaviour_simulation_test_time['date'] = pd.to_datetime(behaviour_simulation_test_time['date']).dt.date.astype(str)
content_simulation_test_time['date'] = pd.to_datetime(content_simulation_test_time['date']).dt.date.astype(str)
content_simulation_test_company['date'] = pd.to_datetime(content_simulation_test_company['date']).dt.date.astype(str)

In [ ]:
# behaviour_simulation_test_company.head() # id date username content media ic
# behaviour_simulation_test_time.head() # id date username content media ic
# content_simulation_test_time.head() # no content
# content_simulation_test_company.head() # no content

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp behaviour_simulation_test_company.csv /content/drive/MyDrive/train_dataset
!cp behaviour_simulation_test_time.csv /content/drive/MyDrive/train_dataset
!cp content_simulation_test_time.csv /content/drive/MyDrive/train_dataset
!cp content_simulation_test_company.csv /content/drive/MyDrive/train_dataset

Mounted at /content/drive
cp: cannot stat 'behaviour_simulation_test_company.csv': No such file or directory
cp: cannot stat 'behaviour_simulation_test_time.csv': No such file or directory
cp: cannot stat 'content_simulation_test_time.csv': No such file or directory
cp: cannot stat 'content_simulation_test_company.csv': No such file or directory
